In [1]:
#import sys
#!{sys.executable} -m pip install request requests_oauthlib

In [2]:
#import sys
#!{sys.executable} -m pip install pygsheets

In [3]:
import os
import pandas as pd
import numpy as np
import urllib
import math
import requests
from requests_oauthlib import OAuth1
#import sympy
#from sympy import *
import pygsheets

#### From Michael Manser - Notebook to import Transferwise Fees for a origination and destination country pair

In [4]:
# Google API stuff (need to set up Google Drive API: https://developers.google.com/sheets/api/quickstart/python)
from __future__ import print_function
#from apiclient.discovery import build
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools


In [5]:
df_orig = pd.read_csv('Transferwise Fee Scrape Setup - Origination.csv')

In [6]:
df_orig[df_orig.currency == 'USD']

,country,currency,pmt_code,pmt_name,speed,txn_size_type,txn_size_denominator
61,USA,USD,ACH,Bank debit (ACH),1,1,1.0
62,USA,USD,REGULAR,Wire transfer,1,1,1.0
63,USA,USD,ADYEN_DEBIT,Debit card,1,1,1.0
64,USA,USD,ADYEN_CREDIT,Credit card,1,1,1.0
65,USA,USD,SWIFT,International wire,2,1,1.0


In [7]:
df_orig = df_orig.loc[(df_orig['country'] == 'USA')]

In [8]:
df_orig

,country,currency,pmt_code,pmt_name,speed,txn_size_type,txn_size_denominator
61,USA,USD,ACH,Bank debit (ACH),1,1,1.0
62,USA,USD,REGULAR,Wire transfer,1,1,1.0
63,USA,USD,ADYEN_DEBIT,Debit card,1,1,1.0
64,USA,USD,ADYEN_CREDIT,Credit card,1,1,1.0
65,USA,USD,SWIFT,International wire,2,1,1.0


In [9]:
#Destination dataframe 
df_dest = pd.read_csv('Transferwise Fee Scrape Setup - Destination.csv')

In [10]:
df_dest.loc[df_dest.index.max()] = ['USD']  #destination currency

In [11]:
df_dest.loc[df_dest.index.max()]

country     USD
currency    USD
Name: 43, dtype: object

In [12]:
# 1. Set send / receive currency pair and list of 
orig_currency = ['USD']
orig_country = ['USA']
dest_currency = ['USD']

#txn_sizes = df_amounts['scrape_amt']

In [13]:
# Make header
authorization_key = 'dad99d7d8e52c2c8aaf9fda788d8acdc'

# Pieces of URL
url_1 = 'https://transferwise.com/api/v1/payment/legacyQuote?amount='
url_2 = '&amountCurrency=source&getNoticeMessages=true&hasDiscount=false&isFixedRate=false&isGuaranteedFixedTarget=false&sourceCurrency='
url_3 = '&targetCurrency='

#txn_sizes = np.array([50,100,200,300,400,500,600,700,800,900,1000,
#                      1500,2000,2500,3000,5000,8000])
txn_sizes = np.array([50,100,200,300,400,500,600,700,800,900,1000,
                      1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,
                      2100,2200,2300,2400,2500,2600,2700,2800,2900,3000])
#nordic_txn_sizes = np.array([499,999,1999,2999,3999,4999,5999,6999,7999,8999,9999,
#                             14999,19999,24999,29999,49999,74999,99999])

In [14]:
df_results = pd.DataFrame(columns=['orig_country','orig_currency','dest_country','dest_currency','pmt_type','delivery_date','txn_amount','total_fee'])

In [15]:
# Loop through
for orig in orig_currency:
    input_2 = str(orig)
    #orig_country = df_orig.loc[df_orig['currency'] == orig,'country'].unique()[0]
    #txn_size_divisor = df_orig.loc[df_orig['currency'] == orig,'txn_size_denominator'].unique()[0]
    txn_size_divisor = 1
    temp_txn_sizes = txn_sizes / txn_size_divisor
    for dest in dest_currency:
        input_3 = str(dest)
        dest_country = df_dest.loc[df_dest['currency'] == dest,'country'].unique()[0]
        for txn in temp_txn_sizes:
            input_1 = str(int(txn)-1)
            full_url = str(url_1 + input_1 + url_2 + input_2 + url_3 + input_3)
            r = requests.get(full_url, headers={'x-authorization-key': authorization_key})
            status = r.status_code
            print(orig, dest, txn, status)
            if status == 200:
                result = r.json()
                for l in range(len(result.get('paymentOptions'))):
                    df_temp = pd.DataFrame({'orig_country':orig_country,
                                            'orig_currency':input_2,
                                            'dest_country':dest_country,
                                            'dest_currency':input_3,
                                            'pmt_type':result.get('paymentOptions')[l].get('type'),
 #                                           'current_date':str(datetime.date.today()),
                                            'delivery_date':result.get('paymentOptions')[l].get('formattedEstimatedDelivery')[3:],
                                            'txn_amount':input_1,
                                            'total_fee':result.get('paymentOptions')[l].get('fee').get('total')},index=[0])
                    df_results = df_results.append(df_temp, ignore_index=True)
                    df_results.head()

USD USD 50.0 200
USD USD 100.0 200
USD USD 200.0 200
USD USD 300.0 200
USD USD 400.0 200
USD USD 500.0 200
USD USD 600.0 200
USD USD 700.0 200
USD USD 800.0 200
USD USD 900.0 200
USD USD 1000.0 200
USD USD 1100.0 200
USD USD 1200.0 200
USD USD 1300.0 200
USD USD 1400.0 200
USD USD 1500.0 200
USD USD 1600.0 200
USD USD 1700.0 200
USD USD 1800.0 200
USD USD 1900.0 200
USD USD 2000.0 200
USD USD 2100.0 200
USD USD 2200.0 200
USD USD 2300.0 200
USD USD 2400.0 200
USD USD 2500.0 200
USD USD 2600.0 200
USD USD 2700.0 200
USD USD 2800.0 200
USD USD 2900.0 200
USD USD 3000.0 200


In [16]:
df_results.to_csv('transferwise_usa_jam_fees.csv',index=False)

In [17]:
df_fees = df_results.copy()
df_fees['txn_amount'] = pd.to_numeric(df_fees['txn_amount']) + 1
df_fees.loc[df_fees['txn_amount'] == 10001,'txn_amount'] = 10000
df_fees_w = df_fees.pivot_table(values='total_fee', index=['orig_country','orig_currency','dest_country','dest_currency','pmt_type'], columns='txn_amount')
df_fees_w.reset_index(inplace=True)

In [18]:
df_fees_w.to_csv('transferwise_USD_JMD_fees.csv',index=False)

In [19]:
df_fees_w.loc[:, ['orig_country', 'orig_currency',  'dest_country', 'dest_currency',
            'pmt_type',              50,             100,             200,
                   300,             400,             500,             600,
                   700,             800,             900,            1000,
                   1100,            1200,            1300,           1400,
                   1500,            1600,            1700,           1800,
                   1900,            2000,            2100,           2200, 
                   2300,            2400,            2500,           2600,
                   2700,            2800,            2900,           3000]]


txn_amount,orig_country,orig_currency,dest_country,dest_currency,pmt_type,50,100,200,300,400,...,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000
0,USA,USD,USD,USD,ADYEN_CREDIT,2.92,4.75,8.42,12.08,15.74,...,77.97,81.63,85.29,88.95,92.62,96.28,99.94,103.60,107.26,110.92
1,USA,USD,USD,USD,ADYEN_DEBIT,1.73,2.35,3.59,4.82,6.06,...,27.04,28.28,29.51,30.75,31.98,33.22,34.45,35.69,36.92,38.15
2,USA,USD,USD,USD,APPLE_PAY,NaN,NaN,NaN,NaN,NaN,...,27.04,28.28,29.51,30.75,31.98,33.22,34.45,35.69,36.92,38.15
3,USA,USD,USD,USD,BALANCE,1.40,1.40,1.40,1.40,1.40,...,1.40,1.40,1.40,1.40,1.40,1.40,1.40,1.40,1.40,1.40
4,USA,USD,USD,USD,DIRECT_DEBIT,1.30,1.48,1.82,2.17,2.52,...,8.45,8.80,9.15,9.50,9.85,10.19,10.54,10.89,11.24,11.59
5,USA,USD,USD,USD,REGULAR,6.47,6.47,6.47,6.47,6.47,...,6.47,6.47,6.47,6.47,6.47,6.47,6.47,6.47,6.47,6.47
